In [ ]:
from IPython.display import SVG
import svgwrite
import numpy as np
import os
import random
import pandas as pd
from random import randint
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from torch.distributions.categorical import Categorical
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
import warnings
import cmath
from svgpathtools import *
import math

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
paths, attributes, svg_attributes = svg2paths2('Minisample/leaves.svg')

In [ ]:
len(paths)

In [ ]:
n_seg = 152 #produces equally sized vectors
ts = np.array(range(n_seg+1))*1/n_seg
figures = []
for path in paths:
    figures.append([[path.point(t).real,path.point(t).imag] for t in ts])
training_data = torch.tensor(figures,dtype=torch.float,device=cuda0)
training_data = training_data[:,1:,:] - training_data[:,:-1,:] #OPTIONAL
training_data = training_data.permute((0,2,1))

In [ ]:
n_seg_rep = 16 #produces equally sized vectors
ts_rep = np.array(range(n_seg_rep+1))*1/n_seg_rep
figures_rep = []
for path in paths:
    figures_rep.append([[path.point(t).real,path.point(t).imag] for t in ts_rep])
training_data_rep = torch.tensor(figures_rep,dtype=torch.float,device=cuda0)
training_data_rep = training_data_rep[:,1:,:] - training_data_rep[:,:-1,:] #OPTIONAL
training_data_rep = training_data_rep.permute((0,2,1))

In [ ]:
x_batches = torch.split(training_data,50)
x_batches_rep = torch.split(training_data_rep,50)

In [ ]:
class CNNArtist(nn.Module):

    def __init__(self): #, dropout=0.1
        super(CNNArtist, self).__init__()
    
        self.encoder = nn.Sequential( # We may add pooling as well, though works perfectly without it 
            nn.Conv1d(2, 2, kernel_size=5,stride=3), #ch: 6, stride=2 
            nn.LeakyReLU(True), 
            nn.Conv1d(2, 2, kernel_size=5,stride=3), #ch: 16 play with those hyperparameters
            nn.LeakyReLU(True))
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=3),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=3),
            nn.LeakyReLU(True),
            nn.Linear(152,152)) 

    def forward(self, x):
        latent = self.encoder(x)
        return latent,self.decoder(latent)

In [ ]:
model = CNNArtist().to(cuda0)
mse_loss = nn.MSELoss()
#lr = 0.5 # learning rate
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)
for epoch in range(6000):
    total_loss = 0
    for i,batch in enumerate(x_batches):
        optimizer.zero_grad()
        latent,out = model(batch)
        #print(coordinates_sigmasq)
        latent_loss = mse_loss(latent,x_batches_rep[i])
        loss = mse_loss(out,batch)
        latent_loss.backward(retain_graph=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1) #what param to set here? (grads are exploding without cliping)
        optimizer.step()
        total_loss += loss.item()
    if epoch % 1000 == 0:
        print(total_loss) #with 10 epochs 7720 2.36    

In [ ]:
s = 0 #0.006
latent,out = model(training_data[s:s+1,:,:])

In [ ]:
flip_x_transform = torch.tensor([[-1.0,0.0],[0.0,1.0]],device=cuda0)
flip_y_transform = torch.tensor([[1.0,0.0],[0.0,-1.0]],device=cuda0)

In [ ]:
x_latent = torch.unsqueeze(torch.mm(latent[0,:,:].T,flip_x_transform).T,0)
y_latent = torch.unsqueeze(torch.mm(latent[0,:,:].T,flip_y_transform).T,0)

In [ ]:
latent = latent + torch.torch.rand((1, 2, 16),device=cuda0)*2

In [ ]:
out = model.decoder(latent)
#out = latent
#out = training_data[s:s+1,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_conv.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv.svg') 

In [ ]:
out = model(training_data[5:6,:,:])
#out = x_batches[0][4:5,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_conv.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv.svg') 

In [ ]:
out = model(training_data[6:7,:,:])
#out = x_batches[0][4:5,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_conv.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv.svg') 